<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/ethosGove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch
import numpy as np
import pandas as pd

In [2]:
!pip install gensim

In [3]:
import gensim.downloader as api

model = api.load('glove-twitter-25')
# model.most_similar('awesome', topn=5).type()

[=====================================-------------] 74.4% 78.0/104.8MB downloaded


In [4]:
def get_synonyms(word, n):
  try:
    words_list=model.most_similar(word, topn=n)
    return words_list  
  except Exception as e:
    return None
  

In [5]:
get_synonyms('awesome',3)

[('amazing', 0.9687871932983398),
 ('best', 0.9600659608840942),
 ('fun', 0.9331520795822144)]

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
import random
from random import sample
random.seed(17)
def synonym_replacement(text, n):
    text=str(text)
    text = text.split()

    random_word_list = list(set([word for word in text if word not in stopwords.words('english')]))
    random.shuffle(random_word_list)
    random_word_list = [word for word in random_word_list if get_synonyms(word,1)]
    try:
      raplace_words=sample(random_word_list,n)
    except Exception as e:
      raplace_words=random_word_list
    texts=[]
    for random_word in raplace_words:
        synonyms_word_list = get_synonyms(random_word,3)
        if synonyms_word_list:
            # synonym = random.choice(synonyms_word_list)
            synonym = synonyms_word_list[0]
            new_words = [synonym[0] if word == random_word else word for word in text]
        texts.append(' '.join(new_words))

    return texts

In [8]:
synonym_replacement("She should ask a few native Americans what their take on this is", 3)

['She should whats a few native Americans what their take on this is',
 'She should ask a few african Americans what their take on this is',
 'She should ask a few native Americans what their give on this is']

In [9]:

!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git
!pip install -qq transformers

Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 184 (delta 101), reused 107 (delta 46), pack-reused 0
Receiving objects: 100% (184/184), 12.63 MiB | 8.04 MiB/s, done.
Resolving deltas: 100% (101/101), done.
     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 3.2MB 27.3MB/s 
     |████████████████████████████████| 890kB 36.1MB/s 


In [11]:

data_path='Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'
df = pd.read_csv(data_path, delimiter=';')
clean_data=[]
for txt in df.comment:
  clean_data.append(txt)
df['tweet']=clean_data
def to_category(score):
  if score <= 0.3:
    return 'NOT'
  elif score>0.7:
    return 'OFF'

  

df['subtask_a']=df.isHate.apply(to_category)
df.dropna(axis=0, how='any', inplace=True)

In [12]:
df

,comment,isHate,tweet,subtask_a
0,You should know women's sports are a joke,1.0,You should know women's sports are a joke,OFF
1,You look like Sloth with deeper Down’s syndrome,1.0,You look like Sloth with deeper Down’s syndrome,OFF
2,You look like Russian and speak like Indian. B...,1.0,You look like Russian and speak like Indian. B...,OFF
3,"Women deserve to be abused, I guess.",1.0,"Women deserve to be abused, I guess.",OFF
4,Women are made for making babies and cooking d...,1.0,Women are made for making babies and cooking d...,OFF
...,...,...,...,...
993,From the midnight sun where the hot springs blow,0.0,From the midnight sun where the hot springs blow,NOT
994,Don't say I'm not your type,0.0,Don't say I'm not your type,NOT
995,And therefore never send to know for whom the...,0.0,And therefore never send to know for whom the...,NOT
996,And I can't stand another day,0.0,And I can't stand another day,NOT


In [13]:
import pandas as pd
df2 = pd.DataFrame({
    'id':[],
    'tweet':[],
    'subtask_a':[],
})
df2 = df2.set_index('id')

In [14]:
for index, row in df.iterrows():
    tweets=synonym_replacement(row['tweet'],3)    
    i=0
    for tweet in tweets:

      id=int(index*10+i+1)# 输出每行的索引值
      df2=df2.append([{'id':id,'tweet':tweets[i],'subtask_a':row['subtask_a']}],ignore_index=False)
      i+=1

In [16]:
df2 = df2.set_index('id')

In [17]:
df2.head()

,tweet,subtask_a
id,,
1.0,You should think women's sports are a joke,OFF
2.0,You should know women's sports are a jokes,OFF
3.0,You should know women's football are a joke,OFF
11.0,You look like Sloth with emotions Down’s syndrome,OFF
12.0,You look think Sloth with deeper Down’s syndrome,OFF


In [ ]:
df2 = df2.set_index('id')

In [18]:
df2.to_csv('ethos_glove.csv')

,tweet,subtask_a,id
0,@USER She should ask a few african Americans w...,0.0,0.0


In [ ]:
df2

,tweet,subtask_a
id,,
864261,@USER She should ask a few african Americans w...,OFF
864262,@USER She should ask a few native Americans wh...,OFF
864263,@USER She should whats a few native Americans ...,OFF


,tweet,subtask_a
id,,
0.0,@USER She should ask a few native Americans wh...,[a]
1.0,@USER She should ask a few native Americans wh...,[a]
2.0,@USER She should ask a few african Americans w...,[a]
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should ask a few african Americans w...,OFF
864263.0,@USER She should whats a few native Americans ...,OFF
